### 前言

本篇主要介绍end-to-end的LLMops流程中的SFT微调->发布->推理流程，使用的SDK版本为0.1.0。建议提前熟悉预测服务相关SDK功能作为前置知识。

In [6]:
import qianfan


## 大模型调优
千帆平台支持SFT/RLHF两种方法进行模型优化，当前SDK已支持对SFT训练微调任务的创建和管理。
SFT 相关操作使用“安全认证/Access Key”中的 Access Key ID 和 Secret Access Key 进行鉴权，无法使用获取Access Token的方式鉴权，相关 key 可以在百度智能云控制台中安全认证获取，详细流程可以参见文档。
鉴权方式除`命名`外，使用方法与预测功能使用的AK 与 SK 方式相同，提供如下三种方式：

- 通过`环境变量`传递（作用于全局，优先级最低）
- 通过`内置函数`传递（作用于全局，优先级大于环境变量）
- 通过`调用接口`时传递（仅作用于该请求，优先级最高）

In [ ]:
# 通过环境变量传递（作用于全局，优先级最低）
import os
os.environ["QIANFAN_ACCESS_KEY"] = "..."
os.environ["QIANFAN_SECRET_KEY"] = "..."

# 通过内置函数传递（作用于全局，优先级大于环境变量）
# import qianfan
# qianfan.AccessKey("...")
# qianfan.SecretKey("...")

# 调用相关接口时传递（仅作用于该请求，优先级最高）
# import qianfan
# task = qianfan.FineTune.create_task(ak="...", sk="...")

In [2]:

import os
os.environ["QIANFAN_ACCESS_KEY"] = 'your_iam_ak'
os.environ["QIANFAN_SECRET_KEY"] = "your_iam_sk"

目前千帆平台支持如下 SFT 相关操作：
* 创建训练任务
* 创建任务运行
* 获取任务运行详情
* 停止任务运行

### 创建SFT任务

创建训练任务需要提供任务名称`name`和任务描述`description`，返回结果在`result`字段中，具体字段与API 文档一致。

In [4]:
# 创建任务
resp = qianfan.FineTune.create_task(name="test_sdk_codeh22j", description="forsdktest1")


In [5]:
# 获取任务ID
task_id = resp["result"]["id"]
task_id

12733

In [2]:
task_id = 12698


### 创建任务运行

创建任务运行需要提供该次训练的详细配置，例如模型版本（`trainType`）、数据集(`trainset`)等等，且不同模型的参数配置存在差异，具体参数可以参见API 文档。

In [ ]:
# 创建任务运行，具体参数可以参见 API 文档
create_job_resp = qianfan.FineTune.create_job({
    "taskId": task_id,
    "baseTrainType": "ERNIE-Bot-turbo",
    "trainType": "ERNIE-Bot-turbo-0725",
    "trainMode": "SFT",
    "peftType": "LoRA",
    "trainConfig": {
        "epoch": 4,
        "learningRate": 0.00002,
        "batchSize": 4,
        "maxSeqLen": 4096
    },
    "trainset": [
        {
            "type": 1,
            "id": 12563
        }
    ],
    "trainsetRate": 20
})

In [9]:
create_job_resp

QfResponse(code=200, headers={'Date': 'Thu, 26 Oct 2023 11:36:06 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '44', 'tracecode': '21660437462240896522102619', 'Set-Cookie': 'BAIDUID=4CD96F03A434BCF8598B88943EFC0773:FG=1; expires=Fri, 25-Oct-24 11:36:06 GMT; max-age=31536000; path=/; domain=.baidu.com; version=1', 'P3P': 'CP=" OTI DSP COR IVA OUR IND COM "', 'X-Bce-Request-Id': '859a7cd8-30bd-456d-8b24-ed07840b4a57', 'X-Bce-Gateway-Region': 'GZ'}, body={'log_id': '3898732436', 'result': {'id': 6506}}, image=None)

In [38]:
import time

job_id = create_job_resp["result"]["id"]
while True:
    job_status_resp = qianfan.FineTune.get_job(task_id=task_id, job_id=job_id)
    job_status = job_status_resp["result"]["trainStatus"]
    print("job status:", job_status)
    if job_status != 'RUNNING':
        break
    time.sleep(60)

job status: FINISH


### 发布模型

In [57]:
sft_task_publish_resp = qianfan.Model.publish(is_new=True, model_name="test_sdk_hj1", version_meta={
    "taskId": task_id, "iterationId": job_id
})
# 获取model_id and version
model_id = sft_task_publish_resp["result"]["modelId"]
model_version = sft_task_publish_resp["result"]["version"]
print("model_id:", model_id)
print("model_version", model_version)

model_id: 7521
model_version 1


In [62]:
# 获取模型版本信息：
model_version_list = qianfan.Model.list(model_id=model_id)
model_version_id: int = 0 
for m in model_version_list["result"]["modelVersionList"]:
    if m["modelId"] == model_id and m["version"] == model_version:
        model_version_id = m["modelVersionId"]
if model_version_id == 0:
    raise ValueError("not model version")
print("model_version_id", model_version_id)

model_version_id 9302


### 发布模型服务

In [ ]:

g = qianfan.Service.create(
    model_id = model_id, 
    iteration_id = model_version_id, 
    name="testsebqf1", 
    uri="sdkqf1",
    replicas=1, 
    pool_type=2
)


In [25]:
svc_id = g["result"]["serviceId"]
svc_id

3803

In [28]:
# 查看模型服务状态
resp = qianfan.Service.get(id = svc_id)
sft_model_endpoint=resp["result"]["uri"]
print("sft_model_endpoint:", sft_model_endpoint)

sft_model_endpoint: t0b2osh7_sdktttt1


### 访问SFT模型服务


In [33]:
os.environ["QIANFAN_AK"] = "your_app_ak"
os.environ["QIANFAN_SK"] = "your_app_sk"

In [39]:
chat_comp = qianfan.ChatCompletion(endpoint=sft_model_endpoint)
msgs = qianfan.Messages()
msgs.append(message="你好，你是谁？", role=qianfan.QfRole.User)
chat_resp = chat_comp.do(messages=msgs)
chat_resp


QfResponse(code=200, headers={'Access-Control-Allow-Headers': 'Content-Type', 'Access-Control-Allow-Origin': '*', 'Appid': '26217442', 'Connection': 'keep-alive', 'Content-Encoding': 'gzip', 'Content-Type': 'application/json; charset=utf-8', 'Date': 'Thu, 26 Oct 2023 12:12:32 GMT', 'P3p': 'CP=" OTI DSP COR IVA OUR IND COM "', 'Server': 'Apache', 'Set-Cookie': 'BAIDUID=7FBBE2E6B5D83AB3C6A3B5F2ABB8E430:FG=1; expires=Thu, 31-Dec-37 23:55:55 GMT; max-age=2145916555; path=/; domain=.baidu.com; version=1', 'Statement': 'AI-generated', 'Vary': 'Accept-Encoding', 'X-Aipe-Self-Def': 'eb_total_tokens:17-id:as-j0ker4w7ar', 'X-Baidu-Request-Id': '9a143024d69419cb6e8ab8bb2d751b8e1000130', 'X-Openapi-Server-Timestamp': '1698322351', 'Content-Length': '222'}, body={'id': 'as-j0ker4w7ar', 'object': 'chat.completion', 'created': 1698322352, 'result': '我是文心一言，是百度研发的。', 'is_truncated': False, 'need_clear_history': False, 'usage': {'prompt_tokens': 5, 'completion_tokens': 12, 'total_tokens': 17}}, image=N

### 总结
至此，你已经通过SFT训练成功的微调出自己的大语言模型，SFT是一个很好的手段，用于针对于特定场景下的语料进行模型特化，以增强模型在某方面的能力，非常适合对于垂直领域内的应用。除了SFT之外，千帆平台还提供了RLHF功能，SDK也将在将来持续跟进LLMOps能力。